In [8]:
import pandas as pd 
from langchain_groq.chat_models import ChatGroq

In [9]:
from config import API
# Groq API and Models 
Groq_Token = API  # Do not share this key with anyone

groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}

In [47]:
## Here we will perform the prompting on all the data
# Accelerometer data
# 561 feature length dataset

## Accelerometer data

from dataset import Accelerometer_X_train,Accelerometer_Y_train,Accelerometer_X_test,Accelerometer_Y_test

filepath = 'UCI HAR Dataset/train/X_train.txt'
Feature_561_X_train = pd.read_csv(filepath, delim_whitespace=True)
filepath = 'UCI HAR Dataset/test/X_test.txt'
Feature_561_X_test = pd.read_csv(filepath, delim_whitespace=True)
filepath = 'UCI HAR Dataset/train/y_train.txt'
Feature_561_Y_train = pd.read_csv(filepath, delim_whitespace=True)
filepath = 'UCI HAR Dataset/test/y_test.txt'
Feature_561_Y_test = pd.read_csv(filepath, delim_whitespace=True)

C:\Users\sriva\AppData\Local\Temp\ipykernel_176656\3753780253.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  Feature_561_X_train = pd.read_csv(filepath, delim_whitespace=True)
C:\Users\sriva\AppData\Local\Temp\ipykernel_176656\3753780253.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  Feature_561_X_test = pd.read_csv(filepath, delim_whitespace=True)
C:\Users\sriva\AppData\Local\Temp\ipykernel_176656\3753780253.py:14: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  Feature_561_Y_train = pd.read_csv(filepath, delim_whitespace=True)
C:\Users\sriva\AppData\Local\Temp\ipykernel_176656\3753780253.py:16: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be remov

In [48]:
print(Accelerometer_X_test.shape)
print(Accelerometer_Y_test.shape)

(54, 500, 3)
(54,)


### Preprocessing dataset to obtain dataset in appropriate format

In [56]:
Accelerometer_X_test_3d = Accelerometer_X_test[:,:20,:]
Accelerometer_X_test_2d = Accelerometer_X_test_3d.reshape(-1,3)
Accelerometer_Y_test_updated = np.repeat(Accelerometer_Y_test, 20)
print(Accelerometer_X_test_2d)
Accelerometer_X_test_2d.shape

[[ 1.293474  -0.3140436 -0.2480047]
 [ 1.258798  -0.2909032 -0.5137133]
 [ 1.258902  -0.3451832 -0.505633 ]
 ...
 [ 0.9335168  0.201874   0.3678763]
 [ 0.9347404  0.1986709  0.3690691]
 [ 0.9329036  0.1934453  0.3717121]]


(1080, 3)

In [57]:
print(Accelerometer_Y_test_updated)
Accelerometer_Y_test_updated.shape

[3 3 3 ... 4 4 4]


(1080,)

In [82]:
dictionary = {
    1 : "Walking",
    2 : "Walking upstairs",
    3 : "Walkimg downstairs",
    4 : "Sitting",
    5 : "Standing",
    6 : "Laying"
}

### ZERO SHOT

In [68]:
## First lets define a function for zero shot prompting
def zero_shot(X):
    # Convert X_pca to DataFrame
    df = pd.DataFrame(X)
    
    zeroshot = []
    for index, row in df.iterrows():
        query = f"""

        I am providing you the accelerometer data as:

        {row.to_dict()}

        Now, based on this accelerometer data and using the dictionary, tell me the type of activity that is performed.

        Here the activities can be as given in the dictionary:

        {dictionary}

        Don't give any explanation just print the answer i.e. the number from the dictionary to which the activity belong
        
        """
        
        model_name = "gemma-7b"  # Choose the model
        llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
        answer = llm.invoke(query)
        
        zeroshot.append(answer.content)
    print(zeroshot)
    return zeroshot

### Using Accelerometer data

In [59]:
zeroshot_response = zero_shot(Accelerometer_X_test_2d)

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',

In [60]:
print(zeroshot_response)

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',

In [63]:
### Quantitavely measure the accuracy of Few-Shot Learning 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

predictions = zeroshot_response
predictions = [int(value) for value in predictions]
true_values = Accelerometer_Y_test_updated  # Extract the single column data
# print("Data type of true_values:", type(true_values[0]))
# print("Data type of predictions:", type(predictions[0]))
assert len(predictions) == len(true_values), "Mismatch between predictions and true labels length"

# Calculate accuracy,precision, recall, and F1-score
accuracy_fewshot = accuracy_score(true_values, predictions)
precision_fewshot = precision_score(true_values, predictions, average='weighted')
recall_fewshot = recall_score(true_values, predictions, average='weighted')
f1_fewshot = f1_score(true_values, predictions, average='weighted')

print(f'Accuracy: {accuracy_fewshot * 100:.2f}%')
print(f'Precision: {precision_fewshot:.2f}')
print(f'Recall: {recall_fewshot:.2f}')
print(f'F1 Score: {f1_fewshot:.2f}')

Accuracy: 17.59%
Precision: 0.05
Recall: 0.18
F1 Score: 0.06


c:\Users\sriva\code\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Using 561 feature length dataset

In [70]:
Feature_Test = Feature_561_X_test[0:500]
print(Feature_Test)
print(Feature_Test.shape)

## As feature length dataset has 561 features which is too large lets apply PCA to get it down
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(Feature_Test)
print("Original shape:", Feature_Test.shape)
print("Reduced shape:", X_pca.shape)


     2.5717778e-001  -2.3285230e-002  -1.4653762e-002  -9.3840400e-001  \
0          0.286027        -0.013163        -0.119083        -0.975415   
1          0.275485        -0.026050        -0.118152        -0.993819   
2          0.270298        -0.032614        -0.117520        -0.994743   
3          0.274833        -0.027848        -0.129527        -0.993852   
4          0.279220        -0.018620        -0.113902        -0.994455   
..              ...              ...              ...              ...   
495        0.284727        -0.009262        -0.113091        -0.988795   
496        0.275832        -0.015331        -0.105358        -0.993626   
497        0.272315        -0.015413        -0.107014        -0.995763   
498        0.270990        -0.013260        -0.114997        -0.987701   
499        0.264218        -0.012400        -0.092979        -0.980132   

     -9.2009078e-001  -6.6768331e-001  -9.5250112e-001  -9.2524867e-001  \
0          -0.967458        -0.94495

In [71]:
zeroshot_response = zero_shot(X_pca)

['3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '4', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '4', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '4', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '4', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',

In [73]:
### Quantitavely measure the accuracy of Few-Shot Learning 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

predictions = zeroshot_response
predictions = [int(value) for value in predictions]
true_values = Accelerometer_Y_test_updated[0:500]  # Extract the single column data
# print("Data type of true_values:", type(true_values[0]))
# print("Data type of predictions:", type(predictions[0]))
assert len(predictions) == len(true_values), "Mismatch between predictions and true labels length"

# Calculate accuracy,precision, recall, and F1-score
accuracy_fewshot = accuracy_score(true_values, predictions)
precision_fewshot = precision_score(true_values, predictions, average='weighted')
recall_fewshot = recall_score(true_values, predictions, average='weighted')
f1_fewshot = f1_score(true_values, predictions, average='weighted')

print(f'Accuracy: {accuracy_fewshot * 100:.2f}%')
print(f'Precision: {precision_fewshot:.2f}')
print(f'Recall: {recall_fewshot:.2f}')
print(f'F1 Score: {f1_fewshot:.2f}')

Accuracy: 11.40%
Precision: 0.01
Recall: 0.11
F1 Score: 0.02


c:\Users\sriva\code\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### FEW SHOT

In [83]:
def few_shot(X,examples):
    df_pca = pd.DataFrame(X)
    fewshot = []
    for index, row in df_pca.head(500).iterrows():
        query = f"""

        I am providing you the accelerometer data as:

        {row}

        Here the activities can be as given in the dictionary:

        {dictionary}

        In the dictionary, an integer corresponding to the activities is provided. Use those integers to denote your response.

        Now, based on this accelerometer data and using the dictionary, tell me the type of activity that is performed.

        Here are few examples for you

        {examples}

        Use these examples and the values provided to find your prediction of the activity

        Don't give any explaination just print the answer, as integer
        """

        model_name = "gemma-7b"  # Choose the model
        llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
        answer = llm.invoke(query)

        fewshot.append(answer.content)
    
    print(fewshot)
    return fewshot

In [84]:
Acclerometer_examples = {
    "Laying" : [0.1811413,0.7878133,0.581194],
    "Sitting" : [0.9256148,0.1663663,0.3673494],
    "Standing" : [1.018851,-0.123976,0.09792958],
    "Walking" : [1.426164,-0.3624851,0.2789144],
    "Walking Downstairs" : [0.8433239,-0.09248479,-0.05186726],
    "Walking Upstairs" : [0.9599337,-0.2482484,0.02457318] 
}

Fewshot prompting on accelerometer data

In [85]:
fewshot_response = few_shot(Accelerometer_X_test_2d,Acclerometer_examples)

['2', '2', '2', '3', '4', '5', '4', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '4', '2', '2', '2', '4', '2', '2', '4', '2', '4', '2', '4', '2', '2', '2', '2', '4', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '4', '2', '2', '2', '2', '4', '2', '2', '4', '4', '2', '4', '2', '2', '4', '4', '2', '4', '2', '2', '2', '2', '4', '2', '2', '2', '2', '2', '4', '4', '2', '4', '2', '4', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '4', '4', '2', '2', '2', '2', '2', '2', '2', '2', '4', '4', '4', '2', '2', '4', '2', '4', '2', '2', '2', '2', '2', '4', '2', '2', '4', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '4', '2', '2', '2', '2', '2', '2', '4', '4', '2', '2', '2', '2', '2', '2', '2', '4', '2', '2', '2', '2', '2', '2', '4', '2', '2', '4', '4', '4', '2', '2', '4', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '4', '2', '2', '2', '2',

In [89]:
### Quantitavely measure the accuracy of Few-Shot Learning 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

predictions = fewshot_response
predictions = [int(value) for value in predictions]
true_values = Accelerometer_Y_test_updated[0:500]  # Extract the single column data
# print("Data type of true_values:", type(true_values[0]))
# print("Data type of predictions:", type(predictions[0]))
# print(len(predictions))
# print(len(true_values))

assert len(predictions) == len(true_values), "Mismatch between predictions and true labels length"

# Calculate accuracy,precision, recall, and F1-score
accuracy_fewshot = accuracy_score(true_values, predictions)
precision_fewshot = precision_score(true_values, predictions, average='weighted')
recall_fewshot = recall_score(true_values, predictions, average='weighted')
f1_fewshot = f1_score(true_values, predictions, average='weighted')

print(f'Accuracy: {accuracy_fewshot * 100:.2f}%')
print(f'Precision: {precision_fewshot:.2f}')
print(f'Recall: {recall_fewshot:.2f}')
print(f'F1 Score: {f1_fewshot:.2f}')

Accuracy: 19.20%
Precision: 0.11
Recall: 0.19
F1 Score: 0.08


c:\Users\sriva\code\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [90]:
pca = PCA(n_components=0.95)
Feature_Train = Feature_561_X_train[0:500]
X_pca = pca.fit_transform(Feature_Train)
print("Original shape:", Feature_Train.shape)
print("Reduced shape:", X_pca.shape)

Original shape: (500, 561)
Reduced shape: (500, 47)


In [95]:
# Print the first row of the PCA-transformed data
print(X_pca[0])

[-6.52509953e+00  4.45750375e-03  1.71641208e+00  1.02777565e-01
 -4.93595550e-01 -4.93692491e-01 -3.58040312e-01 -2.79326666e-01
 -4.14138781e-01 -8.62714676e-01  3.28920570e-01  1.03233620e+00
  2.50406145e-01 -3.52237581e-01 -5.84499347e-01 -5.01157446e-01
 -3.31320279e-01  5.23150691e-01  2.82992418e-01  3.47216280e-01
  1.52066794e-01  2.62907948e-01  2.54938680e-01 -4.27275706e-01
 -3.70721992e-01  3.83185582e-01  3.62287734e-01  4.26584147e-02
 -1.32446228e-01  1.50072064e-01  5.00771286e-01 -1.05578725e-01
  5.25610636e-01  1.91846157e-01 -5.80593825e-01 -2.57269836e-03
 -5.16932418e-02 -3.74709610e-01  5.94748418e-01  3.11882437e-01
 -2.15033679e-01 -1.23986278e-01  3.07345399e-01  1.78053231e-01
 -3.23094932e-03  2.77650340e-01 -2.15889983e-01]


In [96]:
Feature_561_examples = {
    "Laying" : X_pca[51],
    "Sitting" : X_pca[27],
    "Standing" : X_pca[0],
    "Walking" : X_pca[78],
    "Walking Downstairs" : X_pca[125],
    "Walking Upstairs" : X_pca[150]
}

In [98]:
Feature_Test = Feature_561_X_test[0:500]
print(Feature_Test)
print(Feature_Test.shape)

## As feature length dataset has 561 features which is too large lets apply PCA to get it down
from sklearn.decomposition import PCA
pca = PCA(n_components=47)
X_pca = pca.fit_transform(Feature_Test)
print("Original shape:", Feature_Test.shape)
print("Reduced shape:", X_pca.shape)

     2.5717778e-001  -2.3285230e-002  -1.4653762e-002  -9.3840400e-001  \
0          0.286027        -0.013163        -0.119083        -0.975415   
1          0.275485        -0.026050        -0.118152        -0.993819   
2          0.270298        -0.032614        -0.117520        -0.994743   
3          0.274833        -0.027848        -0.129527        -0.993852   
4          0.279220        -0.018620        -0.113902        -0.994455   
..              ...              ...              ...              ...   
495        0.284727        -0.009262        -0.113091        -0.988795   
496        0.275832        -0.015331        -0.105358        -0.993626   
497        0.272315        -0.015413        -0.107014        -0.995763   
498        0.270990        -0.013260        -0.114997        -0.987701   
499        0.264218        -0.012400        -0.092979        -0.980132   

     -9.2009078e-001  -6.6768331e-001  -9.5250112e-001  -9.2524867e-001  \
0          -0.967458        -0.94495

In [99]:
fewshot_response = few_shot(X_pca,Feature_561_examples)

['3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '2', '3', '3', '2', '3', '2', '3', '3', '3', '3', '2', '3', '2', '3', '2', '2', '3', '3', '3', '3', '3', '3', '3', '3', '2', '2', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '2', '3', '3', '3', '3', '3', '3', '2', '3', '2', '3', '3', '2', '3', '2', '3', '2', '2', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '2', '3', '3', '3', '3', '3', '2', '3', '2', '3', '3', '3', '3', '3', '3', '3', '3', '3', '2', '3', '3', '3', '3', '3', '2', '3', '2', '3', '3', '3', '3', '3', '3', '2', '3', '2', '3', '3', '2', '3', '3', '3', '3', '3', '3', '2', '3', '3', '2', '3', '3', '3', '3', '3', '3', '3', '2', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3',

In [100]:
### Quantitavely measure the accuracy of Few-Shot Learning 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

predictions = fewshot_response
predictions = [int(value) for value in predictions]
true_values = Accelerometer_Y_test_updated[0:500]  # Extract the single column data
# print("Data type of true_values:", type(true_values[0]))
# print("Data type of predictions:", type(predictions[0]))
# print(len(predictions))
# print(len(true_values))

assert len(predictions) == len(true_values), "Mismatch between predictions and true labels length"

# Calculate accuracy,precision, recall, and F1-score
accuracy_fewshot = accuracy_score(true_values, predictions)
precision_fewshot = precision_score(true_values, predictions, average='weighted')
recall_fewshot = recall_score(true_values, predictions, average='weighted')
f1_fewshot = f1_score(true_values, predictions, average='weighted')

print(f'Accuracy: {accuracy_fewshot * 100:.2f}%')
print(f'Precision: {precision_fewshot:.2f}')
print(f'Recall: {recall_fewshot:.2f}')
print(f'F1 Score: {f1_fewshot:.2f}')

Accuracy: 11.80%
Precision: 0.03
Recall: 0.12
F1 Score: 0.04


c:\Users\sriva\code\venv\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Q2
Quantitatively compare the accuracy of Few-Shot Learning with Decision Trees (You may use a subset of the test set if you encounter rate-limiting issues). Which method performs better? Why? [1 marks]

Ans : 

Quantitavely as we found in case of Raw accelerometer accuracy for
Few shot = 19.2%
Decision Trees = 67% 
and in case of Dataset Containing 561 features accuracy for 
Few shot = 
Decision Trees = 85%
So it is clear that Decision Trees performs better
Now the decision tree may perform better because of these reasons as the decision trees doesn't rely on pre-trained models and they can be tailored for our specific data.Also in Few shot  pre-trained model might be over-complex leading to generalisation as the data between activities like sitting laying and standing isn't very distinctive similarly for walking , walking upstairs and walking downstairs , and it may not have been able to generalise properly

## Q3
What are the limitations of Zero-Shot Learning and Few-Shot Learning in the context of classifying human activities based on featurized accelerometer data? [1 marks]

Ans : 
### Limitations of Zero-Shot Learning (ZSL)

Dependence on Semantic Descriptions:

ZSL relies heavily on high-quality semantic embeddings or descriptions to relate known classes to unseen classes. If these descriptions do not accurately capture the relationships between activities, the model's performance can suffer significantly.

Lack of Specificity:

ZSL may struggle with highly specific or nuanced human activities that are not well-represented by the semantic embeddings. For example, subtle differences between "Walking" and "Walking upstairs" might be missed if the semantic information doesn't adequately differentiate them.

Generalization Challenges:

Generalizing from seen to unseen classes can be difficult when the new activities have different characteristics or distributions from the training data. ZSL might not perform well if the unseen activities are significantly different from any of the known activities.

### Limitations of Few-Shot Learning (FSL)

Dependency on Quality of Few Samples:

The effectiveness of FSL depends heavily on the quality and representativeness of the few available samples. If these few examples are noisy, unrepresentative, or biased, the model's ability to generalize can be compromised.

Data Imbalance and Overfitting:

With very few examples, there is a risk of overfitting to those specific instances, particularly if there is a significant imbalance between the number of examples for different classes. This can lead to poor generalization to new, unseen examples of the activity.

Complexity in Implementation:

FSL often requires more complex algorithms, such as meta-learning or fine-tuning strategies, which can be more difficult to implement and may require careful tuning to work effectively.


### Q4
What does the model classify when given input from an entirely new activity that it hasn't seen before? [0.5 mark]

Ans :
The model will try to classify the new input as one of the known classes it was trained on. It will assign the input to the closest class according to its learned decision boundaries, even if the input comes from an entirely different activity. This can result in incorrect or nonsensical classifications because the model doesn't recognize that the input is from a new activity

### Q5
Test the model with random data (ensuring the data has the same dimensions and range as the previous input) and report the results. [0.5 mark]

Ans: 

In [104]:
import random
random_data_accelerometer = [random.random() for _ in range(3)]
#print(random_data_accelerometer)

zero_shot_query =  f"""

        I am providing you the accelerometer data as:

        {random_data_accelerometer}

        Now, based on this accelerometer data and using the dictionary, tell me the type of activity that is performed.

        Here the activities can be as given in the dictionary:

        {dictionary}

        Don't give any explanation just print the answer i.e. the number from the dictionary to which the activity belong nothing else
        
        """

model_name = "gemma-7b"  # Choose the model
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(zero_shot_query)
print(answer)

content='2' response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 193, 'total_tokens': 195, 'completion_time': 0.001142218, 'prompt_time': 0.122706666, 'queue_time': 0.0013563340000000007, 'total_time': 0.123848884}, 'model_name': 'gemma-7b-it', 'system_fingerprint': 'fp_7d8efeb0b1', 'finish_reason': 'stop', 'logprobs': None} id='run-184a2e6e-269e-4d0c-8281-61662968539e-0' usage_metadata={'input_tokens': 193, 'output_tokens': 2, 'total_tokens': 195}


In [106]:
few_shot_query = f"""

        I am providing you the accelerometer data as:

        {random_data_accelerometer}

        Here the activities can be as given in the dictionary:

        {dictionary}

        In the dictionary, an integer corresponding to the activities is provided. Use those integers to denote your response.

        Now, based on this accelerometer data and using the dictionary, tell me the type of activity that is performed.

        Here are few examples for you

        {Acclerometer_examples}

        Use these examples and the values provided to find your prediction of the activity

        Don't give any explaination just print the answer, as integer
        """

model_name = "gemma-7b"  # Choose the model
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(few_shot_query)
print(answer)


content='3' response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 449, 'total_tokens': 451, 'completion_time': 0.002225793, 'prompt_time': 0.303605466, 'queue_time': 0.00052629500000001, 'total_time': 0.305831259}, 'model_name': 'gemma-7b-it', 'system_fingerprint': 'fp_7d8efeb0b1', 'finish_reason': 'stop', 'logprobs': None} id='run-43b979c3-8285-4756-bc8e-20c57df3e3f8-0' usage_metadata={'input_tokens': 449, 'output_tokens': 2, 'total_tokens': 451}
